In [12]:
import pandas as pd
import pandas_gbq
import os

In [3]:
Production_Year = 2025

#-- DEF Full --#

DEF_Sebas = pandas_gbq.read_gbq(f""" 
select 
  BF_Status,
  Booking_File_ID,
  DBF.NAME AS Booking_Group_Name,
  Hotel_ID,
  Hotel_Name,
  CASE 
    WHEN Actuals_OTB = 'Actual'THEN 'Actuals'
    WHEN Actuals_OTB = 'OTB' THEN Actuals_OTB
    ELSE 'NO DATA'
  END AS Actuals_OTB,
  Production_Year_Month,
  BF_Start_Date,
  BF_End_Date,
  Feeder_Market_ID,
  Sum(Total_Revenue_Fin_EUR) as Total_Revenue_Fin_EUR, 
  Sum(Total_Revenue_Gross_EUR) as Total_Revenue_Gross_EUR,
  Sum(Total_Revenue_Net_EUR) as Total_Revenue_Net_EUR,
  Agency_Global_ID,
  Agency_Global_Name,
  Agency_ID,
  Agency_Name,
  Intermediary_Global_ID,
  Intermediary_Global_Name,
  Intermediary_ID,
  Intermediary_Name,
  Company_Global_ID,
  Company_Global_Name,
  Company_ID,
  Company_Name,
  C.COUNTRY_TEXT AS Company_Country,
  Guest_Country_ID,
  Hotel_Country,
from `nh-spit-resevations.Commercial_Mirror.CM_Core_Extended` CM
LEFT JOIN `datahub-minoreuam.mice_leads_slv.DIM_BOOKING_FILE` AS DBF ON CM.Booking_File_ID = DBF.ID
LEFT JOIN (SELECT DISTINCT COUNTRY_ID, COUNTRY_TEXT
      FROM `datahub-minoreuam.master_data_slv.DIM_GEOGRAPHY`) AS C ON CM.Company_Country_ID = C.COUNTRY_ID
where 
  Segment_Name_NH in ('Business Group', 'Meetings & Conference') and
  Production_Year in ('{Production_Year}') and 
  Booking_File_ID is not null and
  Total_Revenue_Fin_EUR>0 
  and
  (Agency_Global_ID in ('1100017010','1002868993','1100028426','1100013516','1100039799','1003498873'))
  AND  BF_Status <> 'NOT in CM Extra'
  AND BF_Status = 'CONFI'
  -- AND Booking_File_ID = 'MB0004431539' --- CHECKEO
group by 
  BF_Status,
  Booking_File_ID,
  Booking_Group_Name,
  Hotel_ID,
  Hotel_Name,
  Actuals_OTB,
  Production_Year_Month,
  BF_Start_Date,
  BF_End_Date,
  Feeder_Market_ID,
  Agency_Global_ID,
  Agency_Global_Name,
  Agency_ID,
  Agency_Name,
  Intermediary_Global_ID,
  Intermediary_Global_Name,
  Intermediary_ID,
  Intermediary_Name,
  Company_Global_ID,
  Company_Global_Name,
  Company_ID,
  Company_Name,
  COUNTRY_TEXT,
  Guest_Country_ID,
  Hotel_Country
"""
,project_id="nh-ops"
)

DEF_Sebas.head()

Downloading: 100%|██████████|


,BF_Status,Booking_File_ID,Booking_Group_Name,Hotel_ID,Hotel_Name,Actuals_OTB,Production_Year_Month,BF_Start_Date,BF_End_Date,Feeder_Market_ID,...,Intermediary_Global_Name,Intermediary_ID,Intermediary_Name,Company_Global_ID,Company_Global_Name,Company_ID,Company_Name,Company_Country,Guest_Country_ID,Hotel_Country
0,CONFI,MB0004068533,BCD MEETINGS #2664430,DEBE.KUFUR,NH Berlin Kurfürstendamm,Actuals,2025-02,2025-02-18,2025-03-05,CH,...,None,WITHOUT_I,None,2023636626,ABBVIE INC,2025009519,ABBVIE SCHWEIZ AG,Switzerland,CH,Germany
1,CONFI,MB0004351754,BCD Swedish Police #3266604,AT09.VIEAI,NH Vienna Airport Conference Center,Actuals,2025-10,2025-10-20,2025-11-04,SE,...,None,WITHOUT_I,None,None,None,WITHOUT_C,None,None,SE,Austria
2,CONFI,MB0004528238,GRUPO BBVA_#3668662,ARCC.FLORA,NH Buenos Aires Florida,Actuals,2025-07,2025-07-01,2025-07-04,AR,...,None,WITHOUT_I,None,2000029380,BANCO BILBAO VIZCAYA ARGENTARIA,2000097787,BANCO BBVA ARGENTINA SA,Argentina,AR,Argentina
3,CONFI,MB0004317388,PHARMA GBT / J&J CVENT #3196925 #3227857,DEHE.WFRAN,nhow Frankfurt,Actuals,2025-06,2025-06-02,2025-06-03,DE,...,CVENT INC,1015278154,CVENT INC,2002984480,JOHNSON AND JOHNSON US CORPORATE HE QUARTERS,2200814209,JOHNSON & JOHNSON MEDICAL GMBH,Germany,None,Germany
4,CONFI,MB0004237808,GBT SWEDEN AB AMEX GBT MEETINGS & EVENTS #302,ITTO.STEFA,NH Collection Torino Santo Stefano,Actuals,2025-09,2025-09-16,2025-10-20,SE,...,None,WITHOUT_I,None,None,None,WITHOUT_C,None,None,None,Italy


In [4]:
DEF_Sebas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6305 entries, 0 to 6304
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   BF_Status                 6305 non-null   object
 1   Booking_File_ID           6305 non-null   object
 2   Booking_Group_Name        6305 non-null   object
 3   Hotel_ID                  6305 non-null   object
 4   Hotel_Name                6305 non-null   object
 5   Actuals_OTB               6305 non-null   object
 6   Production_Year_Month     6305 non-null   object
 7   BF_Start_Date             6305 non-null   dbdate
 8   BF_End_Date               6305 non-null   dbdate
 9   Feeder_Market_ID          6305 non-null   object
 10  Total_Revenue_Fin_EUR     6305 non-null   object
 11  Total_Revenue_Gross_EUR   6305 non-null   object
 12  Total_Revenue_Net_EUR     6305 non-null   object
 13  Agency_Global_ID          6305 non-null   object
 14  Agency_Global_Name      

In [6]:
### ---- SEBAS FORMAT ---- ###

## AMEX ##

df_AMEX = pandas_gbq.read_gbq(f"""
    SELECT
  Production_Year,
  CASE
    WHEN RIGHT(Production_Year_Month,2) = "01" THEN '01. January'
    WHEN RIGHT(Production_Year_Month,2) = "02" THEN '02. February'
    WHEN RIGHT(Production_Year_Month,2) = "03" THEN '03. March'
    WHEN RIGHT(Production_Year_Month,2) = "04" THEN '04. April'
    WHEN RIGHT(Production_Year_Month,2) = "05" THEN '05. May'
    WHEN RIGHT(Production_Year_Month,2) = "06" THEN '06. June'
    WHEN RIGHT(Production_Year_Month,2) = "07" THEN '07. July'
    WHEN RIGHT(Production_Year_Month,2) = "08" THEN '08. August'
    WHEN RIGHT(Production_Year_Month,2) = "09" THEN '09. September'
    WHEN RIGHT(Production_Year_Month,2) = "10" THEN '10. October'
    WHEN RIGHT(Production_Year_Month,2) = "11" THEN '11. November'
    WHEN RIGHT(Production_Year_Month,2) = "12" THEN '12. December'
    ELSE 'NO MONTH'
  END AS Production_Month,
  CASE 
    WHEN Actuals_OTB = 'Actual'THEN 'Actuals'
    WHEN Actuals_OTB = 'OTB' THEN Actuals_OTB
    ELSE 'NO DATA'
  END AS Actuals_OTB,
  Booking_File_ID,
  BF_Status,
  DBF.NAME AS Booking_Group_Name,
  Hotel_Name,
  Hotel_City,
  Hotel_Country,
  -- Agency_Global_ID,
  -- Agency_Global_Name,
  Agency_ID,
  Agency_Name,
  C.COUNTRY_TEXT AS Agency_Country,
  BF_Start_Date,
  BF_End_Date,
  Sum(Total_Revenue_Fin_EUR) as Total_Revenue_Fin_EUR, 
from `nh-spit-resevations.Commercial_Mirror.CM_Core_Extended` CM
LEFT JOIN `datahub-minoreuam.mice_leads_slv.DIM_BOOKING_FILE` AS DBF ON CM.Booking_File_ID = DBF.ID
LEFT JOIN (SELECT DISTINCT COUNTRY_ID, COUNTRY_TEXT
      FROM `datahub-minoreuam.master_data_slv.DIM_GEOGRAPHY`) AS C ON CM.Agency_Country_ID = C.COUNTRY_ID
WHERE Segment_Name_NH in ('Business Group', 'Meetings & Conference') 
  AND Production_Year in ('{Production_Year}') 
  AND Booking_File_ID is not null 
  AND Total_Revenue_Fin_EUR>200 
  AND Agency_Global_ID in ('1100017010') 
  -- OR Intermediary_Global_ID in ('1100017010','1002868993','1100028426','1100013516','1100039799','1003498873'))
  AND  BF_Status <> 'NOT in CM Extra'
  AND BF_Status = 'CONFI'
  AND Agency_ID NOT IN ('1001862502','1012285404')
GROUP BY 
  Production_Year,
  Production_Month,
  Actuals_OTB,
  Booking_File_ID,
  Booking_Group_Name,
  BF_Status,
  Hotel_Name,
  Hotel_City,
  Hotel_Country,
  -- Agency_Global_ID,
  -- Agency_Global_Name,
  Agency_ID,
  Agency_Name,
  C.COUNTRY_TEXT,
  BF_Start_Date,
  BF_End_Date
""")

df_AMEX.head()

Downloading: 100%|██████████|


,Production_Year,Production_Month,Actuals_OTB,Booking_File_ID,BF_Status,Booking_Group_Name,Hotel_Name,Hotel_City,Hotel_Country,Agency_ID,Agency_Name,Agency_Country,BF_Start_Date,BF_End_Date,Total_Revenue_Fin_EUR
0,2025,08. August,Actuals,MB0004542228,CONFI,AMEX_SLB_#3695749,NH Collection Bogotá Teleport Royal,Bogota,Colombia,1100023631,GBT TRAVEL SERVICES COLOMBIA S A S,Colombia,2025-08-25,2025-08-27,10494.562470000
1,2025,09. September,Actuals,MB0004644291,CONFI,INTERMEDIATE STAFF PART 3 FY26 FINLAND,NH Collection Helsinki Grand Hansa,Helsinki,Finland,1100012574,GBT FINLAND OY,Finland,2025-09-23,2025-09-26,6798.157897000
2,2025,07. July,Actuals,MB0004068348,CONFI,DLA PIPER UK LLP#2570652,NH Collection Frankfurt Spin Tower,Frankfurt,Germany,1100032374,AMERICAN EXPRESS GLOBAL BUSINESS TR GBT DEUTSC...,Germany,2025-06-27,2025-06-28,3014.099999000
3,2025,09. September,Actuals,MB0004315866,CONFI,Council of EU #3194877,NH Collection Copenhagen,Copenhagen,Denmark,1014401458,GLOBAL BUSINESS TRAVEL BVBA,Belgium,2025-09-29,2025-10-03,9519.905286540
4,2025,08. August,Actuals,MB0004558887,CONFI,CVENT_JJ_TRAIN THE TRAINER/240825,NH Prague City,Prague,Czech Republic,1100003260,GBT TRAVEL SERVICES UK LTD,United Kingdom,2025-08-23,2025-10-03,311.717938996


In [7]:
df_AMEX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Production_Year        276 non-null    object
 1   Production_Month       276 non-null    object
 2   Actuals_OTB            276 non-null    object
 3   Booking_File_ID        276 non-null    object
 4   BF_Status              276 non-null    object
 5   Booking_Group_Name     276 non-null    object
 6   Hotel_Name             276 non-null    object
 7   Hotel_City             276 non-null    object
 8   Hotel_Country          276 non-null    object
 9   Agency_ID              276 non-null    object
 10  Agency_Name            276 non-null    object
 11  Agency_Country         276 non-null    object
 12  BF_Start_Date          276 non-null    dbdate
 13  BF_End_Date            276 non-null    dbdate
 14  Total_Revenue_Fin_EUR  276 non-null    object
dtypes: dbdate(2), object(13

In [8]:
## Kuoni ##

df_Kuoni = pandas_gbq.read_gbq(f"""
  SELECT
  Production_Year,
  CASE
    WHEN RIGHT(Production_Year_Month,2) = "01" THEN '01. January'
    WHEN RIGHT(Production_Year_Month,2) = "02" THEN '02. February'
    WHEN RIGHT(Production_Year_Month,2) = "03" THEN '03. March'
    WHEN RIGHT(Production_Year_Month,2) = "04" THEN '04. April'
    WHEN RIGHT(Production_Year_Month,2) = "05" THEN '05. May'
    WHEN RIGHT(Production_Year_Month,2) = "06" THEN '06. June'
    WHEN RIGHT(Production_Year_Month,2) = "07" THEN '07. July'
    WHEN RIGHT(Production_Year_Month,2) = "08" THEN '08. August'
    WHEN RIGHT(Production_Year_Month,2) = "09" THEN '09. September'
    WHEN RIGHT(Production_Year_Month,2) = "10" THEN '10. October'
    WHEN RIGHT(Production_Year_Month,2) = "11" THEN '11. November'
    WHEN RIGHT(Production_Year_Month,2) = "12" THEN '12. December'
    ELSE 'NO MONTH'
  END AS Production_Month,
  CASE 
    WHEN Actuals_OTB = 'Actual'THEN 'Actuals'
    WHEN Actuals_OTB = 'OTB' THEN Actuals_OTB
    ELSE 'NO DATA'
  END AS Actuals_OTB,
  Booking_File_ID,
  BF_Status,
  DBF.NAME AS Booking_Group_Name,
  Hotel_Name,
  Hotel_City,
  Hotel_Country,
  -- Agency_Global_ID,
  -- Agency_Global_Name,
  Agency_ID,
  Agency_Name,
  C.COUNTRY_TEXT AS Agency_Country,
  BF_Start_Date,
  BF_End_Date,
  Sum(Total_Revenue_Fin_EUR) as Total_Revenue_Fin_EUR, 
from `nh-spit-resevations.Commercial_Mirror.CM_Core_Extended` CM
LEFT JOIN `datahub-minoreuam.mice_leads_slv.DIM_BOOKING_FILE` AS DBF ON CM.Booking_File_ID = DBF.ID
LEFT JOIN (SELECT DISTINCT COUNTRY_ID, COUNTRY_TEXT
      FROM `datahub-minoreuam.master_data_slv.DIM_GEOGRAPHY`) AS C ON CM.Agency_Country_ID = C.COUNTRY_ID
where Segment_Name_NH in ('Business Group', 'Meetings & Conference')
  AND Production_Year in ('{Production_Year}') 
  AND Booking_File_ID is not null
  AND Total_Revenue_Fin_EUR>200 
  AND Agency_Global_ID in ('1100019663') 
  AND  BF_Status <> 'NOT in CM Extra'
  AND BF_Status = 'CONFI'
GROUP BY 
  Production_Year,
  Production_Month,
  Actuals_OTB,
  Booking_File_ID,
  Booking_Group_Name,
  BF_Status,
  Hotel_Name,
  Hotel_City,
  Hotel_Country,
  -- Agency_Global_ID,
  -- Agency_Global_Name,
  Agency_ID,
  Agency_Name,
  C.COUNTRY_TEXT,
  BF_Start_Date,
  BF_End_Date    
""")

df_Kuoni.head()

Downloading: 100%|██████████|


,Production_Year,Production_Month,Actuals_OTB,Booking_File_ID,BF_Status,Booking_Group_Name,Hotel_Name,Hotel_City,Hotel_Country,Agency_ID,Agency_Name,Agency_Country,BF_Start_Date,BF_End_Date,Total_Revenue_Fin_EUR
0,2025,08. August,Actuals,MB0004308081,CONFI,KUONI*KRAFT FINANCE NORVÈGE,NH Nice,Nice,France,1100019663,KUONI GTS (SCHWEIZ) AG,Switzerland,2025-08-15,2025-08-29,5921.090999000
1,2025,04. April,Actuals,MB0004307637,CONFI,BAYER_KUONI #3160640,NH Collection Sevilla,Seville,Spain,1000055893,JTB VIAJES SPAIN S A U,Spain,2025-04-04,2025-04-14,1458.620002000
2,2025,03. March,Actuals,MB0004297170,CONFI,KUONI #3156195,NH Collection Amsterdam Barbizon Palace,Amsterdam,The Netherlands,1100039623,TRAVEL PLAZA NETHERLANDS BV,Netherlands,2025-03-05,2025-03-15,9446.841771000
3,2025,12. December,Actuals,MB0004431594,CONFI,NOBILIA-WERKE #3448736,NH Napoli Panorama,Naples,Italy,1100019663,KUONI GTS (SCHWEIZ) AG,Switzerland,2025-07-15,2026-01-21,5060.223515000
4,2025,11. November,Actuals,MB0004044140,CONFI,World Cheese Award #2163019,NH Bern The Bristol,Bern,Switzerland,1100019663,KUONI GTS (SCHWEIZ) AG,Switzerland,2025-11-11,2025-11-16,28889.185650523


In [9]:
df_Kuoni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Production_Year        70 non-null     object
 1   Production_Month       70 non-null     object
 2   Actuals_OTB            70 non-null     object
 3   Booking_File_ID        70 non-null     object
 4   BF_Status              70 non-null     object
 5   Booking_Group_Name     70 non-null     object
 6   Hotel_Name             70 non-null     object
 7   Hotel_City             70 non-null     object
 8   Hotel_Country          70 non-null     object
 9   Agency_ID              70 non-null     object
 10  Agency_Name            70 non-null     object
 11  Agency_Country         70 non-null     object
 12  BF_Start_Date          70 non-null     dbdate
 13  BF_End_Date            70 non-null     dbdate
 14  Total_Revenue_Fin_EUR  70 non-null     object
dtypes: dbdate(2), object(13)


In [10]:
## BCD ##

df_BCD = pandas_gbq.read_gbq(f"""
WITH data_Cm AS (                             
  SELECT 
  Production_Year,
  CASE
    WHEN RIGHT(Production_Year_Month,2) = "01" THEN '01. January'
    WHEN RIGHT(Production_Year_Month,2) = "02" THEN '02. February'
    WHEN RIGHT(Production_Year_Month,2) = "03" THEN '03. March'
    WHEN RIGHT(Production_Year_Month,2) = "04" THEN '04. April'
    WHEN RIGHT(Production_Year_Month,2) = "05" THEN '05. May'
    WHEN RIGHT(Production_Year_Month,2) = "06" THEN '06. June'
    WHEN RIGHT(Production_Year_Month,2) = "07" THEN '07. July'
    WHEN RIGHT(Production_Year_Month,2) = "08" THEN '08. August'
    WHEN RIGHT(Production_Year_Month,2) = "09" THEN '09. September'
    WHEN RIGHT(Production_Year_Month,2) = "10" THEN '10. October'
    WHEN RIGHT(Production_Year_Month,2) = "11" THEN '11. November'
    WHEN RIGHT(Production_Year_Month,2) = "12" THEN '12. December'
    ELSE 'NO MONTH'
  END AS Production_Month,
  CASE 
    WHEN Actuals_OTB = 'Actual'THEN 'Actuals'
    WHEN Actuals_OTB = 'OTB' THEN Actuals_OTB
    ELSE 'NO DATA'
  END AS Actuals_OTB,
  Booking_File_ID,
  BF_Status,
  DBF.NAME AS Booking_Group_Name,
  Hotel_Name,
  Hotel_City,
  Hotel_Country,
  -- Agency_Global_ID,
  -- Agency_Global_Name,
  Agency_ID,
  Agency_Name,
  C.COUNTRY_TEXT AS Agency_Country,
  BF_Start_Date,
  BF_End_Date,
  Sum(Total_Revenue_Fin_EUR) as Total_Revenue_Fin_EUR, 
from `nh-spit-resevations.Commercial_Mirror.CM_Core_Extended` CM
LEFT JOIN `datahub-minoreuam.mice_leads_slv.DIM_BOOKING_FILE` AS DBF ON CM.Booking_File_ID = DBF.ID
LEFT JOIN (SELECT DISTINCT COUNTRY_ID, COUNTRY_TEXT
      FROM `datahub-minoreuam.master_data_slv.DIM_GEOGRAPHY`) AS C ON CM.Agency_Country_ID = C.COUNTRY_ID
where Segment_Name_NH in ('Business Group', 'Meetings & Conference') 
  AND Production_Year in ('{Production_Year}')
  AND Booking_File_ID is not null
  -- and (Agency_Global_ID in ('1100028426','1100013516','1100039799','1003498873') OR Agency_ID IN ('1015495598','1100028426','1100021960','1100020621','1003166748')
  -- )
  and (Agency_Name like '%BCD%' or Agency_Global_Name like '%BCD%')
  -- OR Intermediary_Global_ID in ('1100017010','1002868993','1100028426','1100013516','1100039799','1003498873'))
  -- AND  BF_Status <> 'NOT in CM Extra'
  AND BF_Status = 'CONFI'
GROUP BY 
  Production_Year,
  Production_Month,
  Actuals_OTB,
  Booking_File_ID,
  Booking_Group_Name,
  BF_Status,
  Hotel_Name,
  Hotel_City,
  Hotel_Country,
  -- Agency_Global_ID,
  -- Agency_Global_Name,
  Agency_ID,
  Agency_Name,
  C.COUNTRY_TEXT,
  BF_Start_Date,
  BF_End_Date
)
SELECT *
FROM data_Cm
GROUP BY 
  Production_Year,
  Production_Month,
  Actuals_OTB,
  Booking_File_ID,
  Booking_Group_Name,
  BF_Status,
  Hotel_Name,
  Hotel_City,
  Hotel_Country,
  -- Agency_Global_ID,
  -- Agency_Global_Name,
  Agency_ID,
  Agency_Name,
  Agency_Country,
  BF_Start_Date,
  BF_End_Date,
  Total_Revenue_Fin_EUR
HAVING SUM(Total_Revenue_Fin_EUR) > 0
""")

df_BCD.head()

Downloading: 100%|██████████|


,Production_Year,Production_Month,Actuals_OTB,Booking_File_ID,BF_Status,Booking_Group_Name,Hotel_Name,Hotel_City,Hotel_Country,Agency_ID,Agency_Name,Agency_Country,BF_Start_Date,BF_End_Date,Total_Revenue_Fin_EUR
0,2025,05. May,Actuals,MB0004444164,CONFI,3470873 - DESAYUNO RAMASA,NH Mexico City Valle Dorado,Mexico City,Mexico,1002844786,BCD TRAVEL,Mexico,2025-05-08,2025-05-08,2484.123701372
1,2025,08. August,Actuals,MB0004524941,CONFI,BCD_JANSSEN_#3664246,NH Collection Bogotá Teleport Royal,Bogota,Colombia,1015495598,TRAVEL CLUB SAS - BCD TRAVEL,Colombia,2025-08-01,2025-08-03,4488.042977544
2,2025,12. December,Actuals,MB0004730529,CONFI,BCD_BOEHRINGER_#4131934,NH Collection Bogotá Teleport Royal,Bogota,Colombia,1015495598,TRAVEL CLUB SAS - BCD TRAVEL,Colombia,2025-12-02,2025-12-20,737.813910460
3,2025,11. November,Actuals,MB0004670719,CONFI,BCD M&E - #4002565,NH Amsterdam Schiphol Airport,Amsterdam,The Netherlands,1016223815,BCD TRAVEL CZECH REPUBLIC SRO,Czech Republic,2025-11-04,2025-11-12,8361.983942000
4,2025,07. July,Actuals,MB0004505782,CONFI,MERCEDES-BENZ #3618699,NH Savona Darsena,Savona,Italy,1100034340,BCD TRAVEL GERMANY GMBH,Germany,2025-07-18,2025-07-19,1879.090902000


In [11]:
df_BCD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Production_Year        568 non-null    object
 1   Production_Month       568 non-null    object
 2   Actuals_OTB            568 non-null    object
 3   Booking_File_ID        568 non-null    object
 4   BF_Status              568 non-null    object
 5   Booking_Group_Name     568 non-null    object
 6   Hotel_Name             568 non-null    object
 7   Hotel_City             568 non-null    object
 8   Hotel_Country          568 non-null    object
 9   Agency_ID              568 non-null    object
 10  Agency_Name            568 non-null    object
 11  Agency_Country         568 non-null    object
 12  BF_Start_Date          568 non-null    dbdate
 13  BF_End_Date            568 non-null    dbdate
 14  Total_Revenue_Fin_EUR  568 non-null    object
dtypes: dbdate(2), object(13

In [19]:
with pd.ExcelWriter("Test_DEF.xlsx",engine="xlsxwriter") as writer:
    # Full DEF
    DEF_Sebas.to_excel(writer, sheet_name = "DEF",index=False)
    # workbook = writer.book
    worksheet = writer.sheets["DEF"]
    ## Ajustamos el ancho de las columnas ##
    for i, col in enumerate(DEF_Sebas.columns):
        max_len = max(DEF_Sebas[col].astype(str).map(len).max(),len(col))+2
        worksheet.set_column(i,i,max_len)

    # AMEX
    df_AMEX.to_excel(writer, sheet_name = "AMEX",index=False)
    worksheet = writer.sheets['AMEX']
    ## Ajustamos el ancho de las columnas ##
    for i, col in enumerate(df_AMEX.columns):
        max_len = max(df_AMEX[col].astype(str).map(len).max(),len(col))+2
        worksheet.set_column(i,i,max_len)

    # BCD
    df_BCD.to_excel(writer,sheet_name="BCD",index=False)
    worksheet = writer.sheets['BCD']
    ## Ajustamos el ancho de las columnas ##
    for i, col in enumerate(df_BCD.columns):
        max_len = max(df_BCD[col].astype(str).map(len).max(),len(col))+2
        worksheet.set_column(i,i,max_len)

    # Kuoni
    df_Kuoni.to_excel(writer,sheet_name="Kuoni",index=False)
    worksheet = writer.sheets['Kuoni']
    for i, col in enumerate(df_Kuoni.columns):
        max_len = max(df_Kuoni[col].astype(str).map(len).max(),len(col))+2
        worksheet.set_column(i,i,max_len)
